In [8]:
from pydantic import BaseModel
from typing import Optional
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import RDF, RDFS
from pyfuseki import FusekiUpdate
from rdflib.plugins.stores import sparqlstore
from rdflib.namespace import XSD
from datetime import datetime
from src.schemas.settings import Settings
from pysolr import Solr

In [17]:
class BfElement(BaseModel):
    value: str
    label: str

class ResourceProperties(BaseModel):
    content: BfElement
    genreForm: Optional[BfElement] = None

class AdminMetadata(BaseModel):
    creationDate: datetime = None
    status: BfElement
    descriptionConventions: BfElement
    identifiedBy: Optional[str] = None

class Title(BaseModel):
    mainTitle: str
    subtitle: Optional[str] = None

class Contribution(BaseModel):
    term: BfElement
    role: BfElement

class Subject(BaseModel):
    type: str
    lang: str
    term: BfElement

class Classification(BaseModel):
    cdd: str
    cutter: str

class BfWork(BaseModel):
    resourceType: list[BfElement]
    adminMetadata: AdminMetadata
    title: Title
    variantTitle: Optional[list[Title]] = None
    language: list[BfElement]
    genreForm: list[BfElement] = None
    contribution: Optional[list[Contribution]] = None
    subject: Optional[list[Subject]] = None
    classification: Classification
    note: Optional[str] = None
    summary: Optional[str] = None
    tableOfContents: Optional[str] = None



In [18]:
req = {
    "resourceType": [
        {
            "value": "Work",
            "label": "Obra"
        },
        {
            "value": "Text",
            "label": "Texto"
        }
    ],
    "adminMetadata": {
        "status": {
            "value": "http://id.loc.gov/vocabulary/mstatus/n",
            "label": "Novo"
        },
        "descriptionConventions": {
            "value": "http://id.loc.gov/vocabulary/descriptionConventions/aacr",
            "label": "AACr"
        }
    },
    "classification": {
        "cdd": "001",
        "cutter": "P831c"
    },
    "title": {
        "mainTitle": "Conjecturas e refutações"
    },
    "language": [
        {
            "value": "por",
            "label": "Português"
        }
    ],
    "contribution": [
        {
            "term": {
                "value": "https://bibliokeia.com/authority/PersonalName/10",
                "label": "Popper, Karl R."
            },
            "role": {
                "value": "http://id.loc.gov/vocabulary/relators/aut",
                "label": "Autor"
            }
        },
        {
            "term": {
                "value": "https://bibliokeia.com/authority/PersonalName/12",
                "label": "Bath, Sérgio"
            },
            "role": {
                "value": "http://id.loc.gov/vocabulary/relators/trl",
                "label": "Tradutor"
            }
        }
    ],
    "subject": [
        {
            "term": {
                "value": "https://bibliokeia.com/authority/Topic/11",
                "label": "Metodologia"
            },
            "lang": "por",
            "type": "Topic"
        }
    ]
}
request = BfWork(**req)

In [21]:
request.adminMetadata.creationDate

In [36]:
now = datetime.now()  
request.adminMetadata.creationDate = now
request.adminMetadata.creationDate.strftime("%Y-%m-%dT%H:%M:%S")

'2024-03-14T10:35:59'

In [4]:
settings = Settings()

In [22]:
def BfType(g, resource, resourceType):
    for i in resourceType:
        bfType = URIRef(f"http://id.loc.gov/ontologies/bibframe/{i.value}")
        g.add((resource, RDF.type, bfType))
    return g

def BfGenreForm(g, resource, genreForm, BF):
    for i in genreForm:
        uri = URIRef(i.value)
        g.add((uri, RDF.type, BF.GenreForm ))
        g.add((uri, RDFS.label, Literal(i.label)))
        g.add((resource, BF.genreForm, uri))
    return g


def BfAdminMetadata(g, adminMetadata, resource, BF):
    now = datetime.now()  

    bNadminMetadata = BNode()
    g.add((resource, BF.adminMetadata, bNadminMetadata))
    g.add((bNadminMetadata, RDF.type, BF.AdminMetadata ))
    # assigner
    assigner_uri = URIRef(settings.organization_loc_uri)
    g.add((assigner_uri, RDF.type, BF.Organization))
    g.add((assigner_uri, RDFS.label, Literal(settings.organization)))
    g.add((bNadminMetadata, BF.assigner, assigner_uri ))
    # creationDate
    formatted_date = now.strftime("%Y-%m-%d")
    g.add((bNadminMetadata, BF.creationDate, Literal(formatted_date, datatype=XSD.date) ))
    # descriptionConventions
    descriptionConventions = URIRef(adminMetadata.descriptionConventions.value)
    g.add((descriptionConventions, RDF.type, BF.DescriptionConventions ))
    g.add((descriptionConventions, RDFS.label, Literal(adminMetadata.descriptionConventions.label) ))
    g.add((bNadminMetadata, BF.descriptionConventions, descriptionConventions ))
    # generationProcess
    formatted_dateTime = now.strftime("%Y-%m-%dT%H:%M:%S")
    generationProcess = BNode() 
    g.add((bNadminMetadata, BF.generationProcess, generationProcess))
    g.add((generationProcess, RDF.type, BF.GenerationProcess))
    g.add((generationProcess, RDFS.label, Literal(settings.app_name) ))
    g.add((generationProcess, BF.generationDate, Literal(formatted_dateTime, datatype=XSD.dateTime) ))
    # identifiedBy
    identifiedBy =  BNode() 
    g.add((bNadminMetadata, BF.identifiedBy, identifiedBy))
    g.add((identifiedBy, RDF.type, BF.Local))
    g.add((identifiedBy, BF.assigner, assigner_uri))
    g.add((identifiedBy, RDF.value, Literal(adminMetadata.identifiedBy)))
    # status
    status = URIRef(adminMetadata.status.value)
    g.add((status, RDF.type, BF.Status))
    g.add((status, RDFS.label, Literal(adminMetadata.status.label)))
    g.add((bNadminMetadata, BF.status, status))

    return g

def BfTitle(g, resource, BF):
    title = BNode()
    g.add((resource, BF.title, title))
    g.add((title, RDF.type, BF.Title))
    g.add((title, BF.mainTitle, Literal(title.mainTitle)))
    if request.title.subtitle:
        g.add((title, BF.subtitle, Literal(request.title.subtitle)))
    if request.variantTitle:
        for i in request.variantTitle:
            variantTitle = BNode()
            g.add((resource, BF.title, variantTitle))
            g.add((variantTitle, RDF.type, BF.VariantTitle))
            g.add((variantTitle, BF.mainTitle, Literal(i.mainTitle)))
            if i.subtitle:
                g.add((variantTitle, BF.subtitle, Literal(i.subtitle)))

    return g

def BFLanguage(g, resource, BF):
    for i in request.language:
        language = URIRef(f"http://id.loc.gov/vocabulary/languages/{i.value}")
        g.add((language, RDF.type, BF.Language))
        g.add((language, RDFS.label, Literal(i.label)))
        g.add((resource, BF.language, language ))
    return g  

def BFContribution(g, resource, BF):
    for i in request.contribution:
        contribution = BNode()
        g.add((resource, BF.contribution, contribution))
        g.add((contribution, RDF.type, BF.Contribution))
        agent = URIRef(i.term.value)
        g.add((agent, RDF.type, BF.Agent))
        g.add((agent, RDF.type, BF.Person))
        g.add((agent, RDFS.label, Literal(i.term.label)))
        g.add((contribution, BF.agent, agent))
        role = URIRef(i.role.value)
        g.add((role, RDF.type, BF.Role))
        g.add((role, RDFS.label, Literal(i.role.label)))
        g.add((contribution, BF.role, role))

    return g

def BFSubject(g, resource, BF):
    for i in request.subject:
        subject = URIRef(i.term.value)
        g.add((subject, RDF.type, BF.Topic))
        g.add((subject, RDFS.label, Literal(i.term.label, lang=i.lang)))
        g.add((resource, BF.subject, subject))

    return g

def BfClassification(g, resource, classification, BF):
    cdd = BNode()
    g.add((resource, BF.classification, cdd))
    g.add((cdd, RDF.type, BF.ClassificationDdc))
    g.add((cdd, BF.classificationPortion, Literal(classification.cdd)))
    cutter = BNode()
    g.add((resource, BF.classification, cutter))
    g.add((cutter, RDF.type, BF.Classification))
    g.add((cutter, BF.classificationPortion, Literal(classification.cutter)))
    return g



In [24]:
request.adminMetadata

AdminMetadata(creationDate=datetime.datetime(2024, 3, 14, 10, 20, 21, 793341), status=BfElement(value='http://id.loc.gov/vocabulary/mstatus/n', label='Novo'), descriptionConventions=BfElement(value='http://id.loc.gov/vocabulary/descriptionConventions/aacr', label='AACr'), identifiedBy=None)

In [ ]:
def MakeGraphWork(request):

    BF = Namespace("http://id.loc.gov/ontologies/bibframe/")
    identifier = f"https://bibliokeia/works/{request.adminMetadata.identifiedBy}"
    resource = URIRef(identifier)

    g = Graph(identifier=identifier)
    g.bind('bf', BF)
    
    # resourceType
    BfType(g, resource, request.resourceType)
    # AdminMetadata
    g = BfAdminMetadata(g, request.adminMetadata, request, resource, BF) 
    # BfClassification
    g = BfClassification(g, resource, request.classification, BF)
    # Title
    g = BfTitle(g, request, resource, BF)
    # Language
    g = BFLanguage(g, request.language, resource, BF)
    # Contribution
    g = BFContribution(g, request.contribution, resource, BF)
    # Subject
    g = BFSubject(g, request.subject, resource, BF) 
    # GenreForm
    if request.genreForm and request.genreForm.valeu != "":
        g = BfGenreForm(g, resource, request.genreForm, BF)    
 
    graph = g.serialize(format='ttl')
    # g.serialize(destination="work_test.ttl", format='ttl')

    return graph

def MakeSparql(graph, ):
    graph = graph.split("\n\n")
    graph = "\n\n".join(graph[1:])

    graph = f"""
        PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>  
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

        INSERT DATA {{
            GRAPH <https://bibliokeia/works/10> 
            {{
                {graph} }} }} """

    return graph



graph = MakeGraphWork(request)
sparql = MakeSparql(graph)
print(sparql)

In [27]:
fuseki = FusekiUpdate("http://localhost:3030" , 'bk') 
response = fuseki.run_sparql(sparql) 
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [48]:
update_endpoint = 'http://localhost:3030/bk/update'
query_endpoint = 'http://localhost:3030/bk/query'

store = sparqlstore.SPARQLUpdateStore(query_endpoint=query_endpoint, update_endpoint=update_endpoint)
# store.open((query_endpoint, update_endpoint))

In [50]:
# url = 'http://localhost:3030/bk/data?graph=https:%2F%2Fbibliokeia%2Fworks%2F10'
url = "http://id.loc.gov/resources/works/21984730.rdf"
g = Graph()
g.parse(url)

<Graph identifier=Nf8a4447e118c409ab437162e76a2afcd (<class 'rdflib.graph.Graph'>)>

In [37]:
def DocWork(request, work_id):

    work_id = f'work#{work_id}'

    doc = {
        "id": work_id,
        "creationDate": request.adminMetadata.creationDate.strftime("%Y-%m-%dT%H:%M:%S"),
        "type": [i.value for i in request.resourceType],
        # "content": request.content.label,
        "mainTitle": request.title.mainTitle,
        'language': [i.label for i in request.language],
        "subtitle": request.title.subtitle,
        "cdd": request.classification.cdd,
        "cutter": request.classification.cutter,
        "note": request.note,
        "summary": request.summary,
        "tableOfContents": request.tableOfContents,
        # "supplementaryContent": [i.label for i in request.supplementaryContent] if request.supplementaryContent else None,
        # "illustrativeContent": [i.label for i in request.illustrativeContent] if request.illustrativeContent else None,
        # "intendedAudience": [i.label for i in request.intendedAudience] if request.intendedAudience else None,
        # "geographicCoverage": [i.label for i in request.geographicCoverage] if request.geographicCoverage else None,
        "isPartOf": "Work"

    }
    # contribution
    if request.contribution:
        contributions = list()
        for i in request.contribution:
            c = {"id": f"{work_id}/contribution/{i.term.value.split('/')[-1]}",
                 "agent": i.term.value,
                 "label": i.term.label,
                 "role": i.role.value,
                 "roleLabel": i.role.label}
            contributions.append(c)
        doc['contribution'] = contributions

    # subject
    if request.subject:
        subjects = list()
        for i in request.subject:
            s = {"id": f"{work_id}/subject/{i.term.value.split('/')[-1]}",
                 "type": i.type,
                 "uri": i.term.value,
                 "label": i.term.label}
            subjects.append(s)
        doc['subject'] = subjects

    if request.genreForm:
        pass

    responseSolr = solr.add([doc], commit=True)

    return responseSolr

doc = DocWork(request, "teste")

DOC: {'id': 'work#teste', 'creationDate': '2024-03-14T10:35:59', 'type': ['Work', 'Text'], 'mainTitle': 'Conjecturas e refutações', 'language': ['Português'], 'subtitle': None, 'cdd': '001', 'cutter': 'P831c', 'note': None, 'summary': None, 'tableOfContents': None, 'isPartOf': 'Work', 'contribution': [{'id': 'work#teste/contribution/10', 'agent': 'https://bibliokeia.com/authority/PersonalName/10', 'label': 'Popper, Karl R.', 'role': 'http://id.loc.gov/vocabulary/relators/aut', 'roleLabel': 'Autor'}, {'id': 'work#teste/contribution/12', 'agent': 'https://bibliokeia.com/authority/PersonalName/12', 'label': 'Bath, Sérgio', 'role': 'http://id.loc.gov/vocabulary/relators/trl', 'roleLabel': 'Tradutor'}], 'subject': [{'id': 'work#teste/subject/11', 'type': 'Topic', 'uri': 'https://bibliokeia.com/authority/Topic/11', 'label': 'Metodologia'}]}


In [38]:
doc

{'id': 'work#teste',
 'creationDate': '2024-03-14T10:35:59',
 'type': ['Work', 'Text'],
 'mainTitle': 'Conjecturas e refutações',
 'language': ['Português'],
 'subtitle': None,
 'cdd': '001',
 'cutter': 'P831c',
 'note': None,
 'summary': None,
 'tableOfContents': None,
 'isPartOf': 'Work',
 'contribution': [{'id': 'work#teste/contribution/10',
   'agent': 'https://bibliokeia.com/authority/PersonalName/10',
   'label': 'Popper, Karl R.',
   'role': 'http://id.loc.gov/vocabulary/relators/aut',
   'roleLabel': 'Autor'},
  {'id': 'work#teste/contribution/12',
   'agent': 'https://bibliokeia.com/authority/PersonalName/12',
   'label': 'Bath, Sérgio',
   'role': 'http://id.loc.gov/vocabulary/relators/trl',
   'roleLabel': 'Tradutor'}],
 'subject': [{'id': 'work#teste/subject/11',
   'type': 'Topic',
   'uri': 'https://bibliokeia.com/authority/Topic/11',
   'label': 'Metodologia'}]}

In [39]:
solr = Solr(f'{settings.solr}/solr/catalog/', timeout=10)

In [40]:
responseSolr = solr.add([doc], commit=True)
responseSolr

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":82\n  }\n}'

In [13]:
request.adminMetadata.creationDate

'08/03/2024'